In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [30]:
nltk.download('stopwords', quiet=True)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


False

In [17]:
import sys
print(sys.executable)

/Users/lloydbrown/venv/bin/python


In [5]:
df=pd.read_csv("emails.csv", sep=",")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['spam'] = pd.to_numeric(df['spam'], errors='coerce')

In [3]:
#head
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0


In [37]:
#tail
df.tail()

,text,spam
5725,Subject: re : research and development charges...,0
5726,"Subject: re : receipts from visit jim , than...",0
5727,Subject: re : enron case study update wow ! a...,0
5728,"Subject: re : interest david , please , call...",0
5729,Subject: news : aurora 5 . 2 update aurora ve...,0


In [6]:
#shape
df.shape

(5730, 110)

In [7]:
#info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5730 entries, 0 to 5729
Columns: 110 entries, text to Unnamed: 109
dtypes: float64(1), object(109)
memory usage: 4.8+ MB


In [47]:
#missing values
missing_values = df.isnull().sum()

print(missing_values)

text    0
spam    4
dtype: int64


In [41]:
print(df.columns.tolist())

print(df['spam'].unique())
print(df['spam'].dtype)

['text', 'spam']
['1' '0' nan
 ' its termination would not  have such a phenomenal impact on the power situation .  however '
 ' mr suresh prabhu ']
object


In [45]:
# mean imputation
emails_mean_imputed = df.copy()
#emails_mean_imputed['spam'].fillna(df['spam'].mean(), inplace=True)
emails_mean_imputed['spam'] = emails_mean_imputed['spam'].fillna(df['spam'].mean())

In [46]:
emails_mean_imputed

,text,spam
0,Subject: naturally irresistible your corporate...,1.0
1,Subject: the stock trading gunslinger fanny i...,1.0
2,Subject: unbelievable new homes made easy im ...,1.0
3,Subject: 4 color printing special request add...,1.0
4,"Subject: do not have money , get software cds ...",1.0
...,...,...
5725,Subject: re : research and development charges...,0.0
5726,"Subject: re : receipts from visit jim , than...",0.0
5727,Subject: re : enron case study update wow ! a...,0.0
5728,"Subject: re : interest david , please , call...",0.0


In [10]:
#duplicates
df.duplicated().sum()

np.int64(33)

In [48]:
#garbage values
for i in df.select_dtypes(include="object").columns:
    print(df[i].value_counts())
    print("***"*10)

text
Subject: re : contact info  glenn ,  please , contact rudi zipter to set up an interview with him and david port .  vince  " glenn darrah " on 04 / 25 / 2001 04 : 27 : 03 pm  please respond to gdarrah @ bigfoot . com  to : vince . j . kaminski @ enron . com  cc :  subject : contact info  vincent ,  congratulations on spearheading the mind ' s eye madness event - it looks like  quite a success . my biggest disappointment is that i am leaving enron this  week and have been too busy to participate as much as i would like . i have  had continued interest in the work and presentations that amy oberg has  done , so would have really enjoyed the workshop yesterday . i am still  considering doing some or all of the motorcade friday , however that may  work .  separately , thanks for the update in the lobby today on the enterprise risk  project . please keep in touch with me . as i mentioned , my redeployment  period ends tomorrow ( april 26 ) , but i currently do not have an outside job  

In [49]:
df['spam'].describe()

count    5726.000000
mean        0.238910
std         0.426455
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: spam, dtype: float64

In [9]:
df['finders'] = df['text'].apply(lambda x: re.findall(r'\b(and|if|or)\b', str(x)))

print(df[['text', 'finders']])

                                                   text  \
0     Subject: naturally irresistible your corporate...   
1     Subject: the stock trading gunslinger  fanny i...   
2     Subject: unbelievable new homes made easy  im ...   
3     Subject: 4 color printing special  request add...   
4     Subject: do not have money , get software cds ...   
...                                                 ...   
5725  Subject: re : research and development charges...   
5726  Subject: re : receipts from visit  jim ,  than...   
5727  Subject: re : enron case study update  wow ! a...   
5728  Subject: re : interest  david ,  please , call...   
5729  Subject: news : aurora 5 . 2 update  aurora ve...   

                                                finders  
0                             [and, and, and, and, and]  
1                           [and, or, and, or, and, or]  
2                                            [and, and]  
3                                             [and, or]  
4

In [28]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))

In [29]:
stemmer = PorterStemmer()
corpus = []

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans(' ', ' ', string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/lloydbrown/nltk_data'
    - '/Users/lloydbrown/venv/nltk_data'
    - '/Users/lloydbrown/venv/share/nltk_data'
    - '/Users/lloydbrown/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
